In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp, Trials
import time
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Загрузка данных
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/diabetes.csv')
X = df.drop(columns=['Outcome'])  # Признаки
y = df['Outcome']  # Целевая переменная

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация базовой модели XGBoost
xgb_model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')

In [ ]:
# Определение пространства гиперпараметров для Random Search
params = {
    'max_depth': np.arange(1, 11),  # Максимальная глубина дерева
    'n_estimators': np.arange(1, 1000),  # Количество деревьев в ансамбле
    'learning_rate': [1, 0.1, 0.01, 0.05]  # Темп обучения
}


In [ ]:
# RandomizedSearchCV для поиска гиперпараметров
random_search = RandomizedSearchCV(xgb_model, params, cv=5, random_state=42)

In [ ]:
# Измерение времени выполнения поиска
start_time = time.time()
random_search.fit(X_train, y_train)
end_time = time.time()

# Получение лучших параметров
best_params_random = random_search.best_params_
print("Лучшие параметры Random Search:", best_params_random)
print("Время поиска Random Search: {:.2f} секунд".format(end_time - start_time))

Лучшие параметры Random Search: {'n_estimators': 984, 'max_depth': 2, 'learning_rate': 0.01}
Время поиска Random Search: 12.93 секунд


In [ ]:
# Определение пространства гиперпараметров для HyperOpt
space = {
    'n_estimators': hp.choice('n_estimators', np.arange(1, 1000)),
    'max_depth': hp.choice('max_depth', np.arange(1, 11)),
    'learning_rate': hp.choice('learning_rate', [1, 0.1, 0.01, 0.05]),
}

In [ ]:
# Определение функции цели для минимизации ошибки
def objective(params):
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, predictions)
    return -accuracy  # Минимизируем отрицательную точность


In [ ]:
# Использование метода TPE для оптимизации
trials = Trials()
start_time = time.time()
best_hyperopt = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)
end_time = time.time()


100%|██████████| 100/100 [00:45<00:00,  2.20trial/s, best loss: -0.7857142857142857]


In [ ]:
# Получение лучших параметров после поиска HyperOpt
best_params_hyperopt = {
    'n_estimators': best_hyperopt['n_estimators'],
    'max_depth': best_hyperopt['max_depth'],
    'learning_rate': [1, 0.1, 0.01, 0.05][best_hyperopt['learning_rate']]  # Получаем реальное значение из индекса
}
print("Лучшие параметры TPE (HyperOpt):", best_params_hyperopt)
print("Время поиска HyperOpt: {:.2f} секунд".format(end_time - start_time))


Лучшие параметры TPE (HyperOpt): {'n_estimators': 147, 'max_depth': 1, 'learning_rate': 0.01}
Время поиска HyperOpt: 45.70 секунд


Параметры, предложенные Random Search и TPE&Hyperopt отличаются, проверим результаты:

In [ ]:
# Модели с лучшими параметрами от Random Search и HyperOpt
best_model_random = XGBClassifier(**best_params_random)
best_model_hyperopt = XGBClassifier(**best_params_hyperopt)

# Обучение и тестирование модели с Random Search
best_model_random.fit(X_train, y_train)
y_pred_random = best_model_random.predict(X_test)
accuracy_random = accuracy_score(y_test, y_pred_random)
print("Точность модели с Random Search: {:.4f}".format(accuracy_random))

# Обучение и тестирование модели с TPE (HyperOpt)
best_model_hyperopt.fit(X_train, y_train)
y_pred_hyperopt = best_model_hyperopt.predict(X_test)
accuracy_hyperopt = accuracy_score(y_test, y_pred_hyperopt)
print("Точность модели с TPE (HyperOpt): {:.4f}".format(accuracy_hyperopt))

# Сравнение результатов
print("\nСравнение методов поиска гиперпараметров:")
print("Точность Random Search: {:.4f}".format(accuracy_random))
print("Точность TPE (HyperOpt): {:.4f}".format(accuracy_hyperopt))

Точность модели с Random Search: 0.7532
Точность модели с TPE (HyperOpt): 0.7468

Сравнение методов поиска гиперпараметров:
Точность Random Search: 0.7532
Точность TPE (HyperOpt): 0.7468


### Вывод
Random Search оказался точнее